In [1]:
import serial
import pandas as pd
from configparser import ConfigParser as cfgParser
from datetime import datetime
import time

configDir='./config/'
settingsPath=configDir+'settings.ini'

In [2]:
#loading information
settings=cfgParser()
settings.read(settingsPath)

PLAIN_OUTPUT_FILE=settings.get('Path','PLAIN_OUTPUT_FILE')
BACKUP_OUTPUT_FILE=settings.get('Path','BACKUP_OUTPUT_FILE')
FINAL_OUTPUT_FILE=settings.get('Path','FINAL_OUTPUT_FILE')
USERS_DICTIONARY_FILE=settings.get('Path','USERS_DICTIONARY_FILE')

SERIAL_PORT_NAME=settings.get('SerialConfig','SERIAL_PORT_NAME')
SERIAL_BAUD_RATE=settings.getint('SerialConfig','SERIAL_BAUD_RATE')

READ_FILE_COMMAND=settings.get('Commands','READ_FILE_COMMAND')
DELETE_FILE_COMMAND=settings.get('Commands','DELETE_FILE_COMMAND')
SET_TIME_COMMAND=settings.get('Commands','SET_TIME_COMMAND')

TEXT_FORMATTER_PREFIX=settings.get('Formatting','PREFIX')
TEXT_FORMATTER_SUFFIX=settings.get('Formatting','SUFFIX')

BASE_ENCODING='utf-8'

usersMap=pd.read_csv(USERS_DICTIONARY_FILE)

In [3]:
#connect to board
serialClient = serial.Serial(SERIAL_PORT_NAME, SERIAL_BAUD_RATE)
def sendCommand(command):
    b = bytes(command, BASE_ENCODING)
    serialClient.write(b)
time.sleep(7)

In [4]:
#sync time

timeCommand=SET_TIME_COMMAND.replace('#year',str(datetime.now().year).rjust(4,'0')).replace('#month',str(datetime.now().month).rjust(2,'0')).replace('#day',str(datetime.now().day).rjust(2,'0')).replace('#hour',str(datetime.now().hour).rjust(2,'0')).replace('#minute',str(datetime.now().minute).rjust(2,'0')).replace('#seconds',str(datetime.now().second).rjust(2,'0')).replace('12OfWeek',str((datetime.now().weekday()+1)%7+1))
sendCommand(timeCommand)
time.sleep(1)

In [5]:
#read data
sendCommand(READ_FILE_COMMAND)
fileReader=True
logData=''
while(fileReader):
    line = str(serialClient.readline().decode(BASE_ENCODING))
    logData+=line
    if line.rfind('LOGFILE_END')>=0:
        fileReader=False
logData=logData.replace(TEXT_FORMATTER_PREFIX,'')
logData=logData.replace(TEXT_FORMATTER_SUFFIX,'')
logData.replace('\r\n','\n')
logData.replace('\r','')
time.sleep(1)


In [6]:
sendCommand(DELETE_FILE_COMMAND)

In [7]:
#backup
f = open(BACKUP_OUTPUT_FILE, "a")
f.write(logData)
f.close()

In [8]:
#savePlain
f = open(PLAIN_OUTPUT_FILE, "w")
f.write('ID,DATE_TIME'+logData)
f.close()

In [9]:
for index, row in usersMap.iterrows():
    logData=logData.replace(row['BOARD_ID'],row['USER_ID'])

In [10]:
f = open(FINAL_OUTPUT_FILE, "w")
f.write('ID,DATE_TIME'+logData)
f.close()